In [2]:
import re
from pathlib import Path
from collections import namedtuple
from itertools import chain
from typing import List, Tuple, Dict, Union, Optional

import bs4
from bs4 import BeautifulSoup as Soup
import pandas as pd
from pprint import pprint

from tqdm.auto import tqdm

In [3]:
Feats = Dict[str, str]
WordDict = Dict[str, Union[str, Feats]]

In [4]:
SYNTAGRUS_DIR = './syntagrus/syntagrus/syntagrus2020'
PARSER = 'lxml-xml'

syntagrus_dir = Path(SYNTAGRUS_DIR)

In [5]:
files = list(syntagrus_dir.glob('**/*.tgt'))

In [49]:
desired_cols = [
    'sent_num_file', 'left_context', 'text', 'right_context',
        'full_analysis', 'sent_pos_chain',
    'has_many_dos', 'has_many_ios',     # whether sentence has many objs with same deprel
    'has_nodetype_in_our_forms', 
    'verb_inf', 'verb',
    'verb_feats',                       # full verb features string
    'verb_has_neg',                     # binary: 1 - there is a "ne", 0 - there isn't
    'DO_group', 'IO_group',             # full NPs/PPs
        'DO_had_appos', 'IO_had_appos',
        'DO_has_relcl', 'IO_has_relcl',
            'DO_has_wh_relcl', 'IO_has_wh_relcl',
            'DO_has_participle', 'IO_has_participle',
    'DO_depth', 'IO_depth',
    'DO_length', 'IO_length', 
    'DO_pos_feat_chain', 'IO_pos_feat_chain',
    'DO',                               # word forms
    'DO_pos', 'DO_lemma',    
        'DO_animacy', 'DO_case',
        'DO_number',
    'IO',
    'IO_pos', 'IO_head', 'IO_head_pos', 'IO_lemma',
        'IO_animacy', 'IO_case',
        'IO_number',
    'order',                            # in terms of V/IO/DO
    ]
Res = namedtuple('res', desired_cols)

In [7]:
PRON_STR = (
    'я мы ты вы он она оно они '
    'себя '
    'свой мой наш твой ваш его её их '
    'кто что где откуда зачем когда сколько какой чей который каковой каков '
    'то тот этот столько такой таков сей оный '
    'всяк всякий каждый сам самый любой иной другой весь все '
    'никто ничто некого нечего нигде ниоткуда неоткуда незачем никогда нисколько никакой ничей '
    'некто нечто некий некоторый несколько '
)

PRONS = set(PRON_STR.split())


PRON_ONLY_MODIFIER_STR = 'кое- -либо -нибудь -то было'
PRON_ONLY_MODIFIERS = set(PRON_ONLY_MODIFIER_STR.split())

In [8]:
lemmas = 'кто бы то ни было, кое-кто, кому-нибудь, что-то, что-либо, кем бы то ни было, былое, коек, либонуть'

for lemma in lemmas.split(','):
    has_modif = any(re.search(rf'\b{modif}\b', lemma) for modif in PRON_ONLY_MODIFIERS)
    print(f'{lemma} {has_modif}')

кто бы то ни было True
 кое-кто True
 кому-нибудь True
 что-то True
 что-либо True
 кем бы то ни было True
 былое False
 коек False
 либонуть False


In [9]:
PRON_STR

'я мы ты вы он она оно они себя свой мой наш твой ваш его её их кто что где откуда зачем когда сколько какой чей который каковой каков то тот этот столько такой таков сей оный всяк всякий каждый сам самый любой иной другой весь все никто ничто некого нечего нигде ниоткуда неоткуда незачем никогда нисколько никакой ничей некто нечто некий некоторый несколько '

In [10]:
common_features = {
    'is_composite': 'сл',
}

nominal_features = {
    'gender': 'муж жен сред',
    'number': 'ед мн',
    'case': 'им род парт дат вин твор пр местн зв',
    'animacy': 'неод од',
}

adjectival_features = {
    'brevity': 'кр',
}

comparative_features = {
    'comp_type': 'смяг',
    'comp': 'срав прев',
}

verb_format = {
    'representation': 'инф прич деепр',
    'mood': 'изъяв пов',
    'aspect': 'несов сов',
    'tense': 'непрош прош наст',
    'person': '1-л 2-л 3-л',
    'voice': 'страд',
    **adjectival_features,
    **nominal_features,
}

noun_format = {
    **nominal_features,
}

adjective_format = {
    **comparative_features,
    **adjectival_features,
    **nominal_features
}

adverb_format = {
    **comparative_features,
}

numeral_format = {
    **nominal_features
}

def get_pos_to_feats_format():
    pos_to_feats_format = {
        'V': dict(verb_format), 'S': dict(noun_format), 'A': dict(adjective_format),
        'ADV': dict(adverb_format), 'NUM': dict(numeral_format), 'COM': {},
    }
    
    for pos in pos_to_feats_format.keys():
        pos_to_feats_format[pos].update(common_features)

    
    for pos, pos_format in pos_to_feats_format.items():
        for param, values in pos_format.items():
            pos_format[param] = values.split()
            
    return pos_to_feats_format

In [11]:
def is_sentence_with_accdo_io(
    tag: bs4.element.Tag,
    first_object_possible_roles: List[str] = ['1-компл'],
    second_object_possible_roles: List[str] = ['2-компл'],
) -> bool:
    is_sentence = tag.name == 'S'
    if not is_sentence:
        return False
    
    sent_str = str(tag)
    
    first_object_rel_str = (
        f'{first_object_possible_roles[0]}' if len(first_object_possible_roles) == 1
        else '(?:{})'.format('|'.join(second_object_possible_roles))
    )
    
    first_object_pat = re.compile(
        rf'^<W DOM=".*" FEAT="(?:\w+ )*?ВИН(?: \w+)*?".*LINK="{first_object_rel_str}">.*?$',
        re.MULTILINE
    )
    first_object = re.search(
        first_object_pat,
        sent_str)
    if not first_object:
        return False
    
    second_object_rel_str = (
        f'{second_object_possible_roles[0]}' if len(second_object_possible_roles) == 1
        else '(?:{})'.format('|'.join(second_object_possible_roles))
    )
    second_object = re.search(rf'LINK="{second_object_rel_str}"', sent_str)
    if not second_object:
        return False
    else:
        return True

In [12]:
def get_soup(filename: Path) -> bs4.BeautifulSoup:
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
        soup = Soup(text, PARSER)
    
    return soup

In [13]:
def find_sentences_do_io(
    body: bs4.element.Tag,
    is_sentence_with_do_io_kwargs = {}
) -> List[bs4.element.Tag]:
    
    sents = body.find_all(
        lambda tag: is_sentence_with_accdo_io(tag, **is_sentence_with_do_io_kwargs),
        recursive=False
    )
    
    return sents

In [14]:
file = syntagrus_dir / Path('2019/baden-baden.tgt')

In [15]:
file_soup = get_soup(file).find('body')

In [16]:
sents = file_soup.find_all('S')

In [17]:
ss = find_sentences_do_io(file_soup)

In [18]:
ss

[<S ID="9">
 <W DOM="2" FEAT="S ЕД МУЖ ИМ ОД" ID="1" KSNAME="ОН" LEMMA="ОН" LINK="предик">Он</W>
 <W DOM="_root" FEAT="V НЕСОВ ИЗЪЯВ ПРОШ ЕД МУЖ" ID="2" LEMMA="ЧУВСТВОВАТЬ">чувствовал</W>
 <W DOM="2" FEAT="S ЕД МУЖ ВИН ОД" ID="3" LEMMA="СЕБЯ" LINK="1-компл">себя</W>
 <W DOM="2" FEAT="ADV" ID="4" LEMMA="ЗДЕСЬ" LINK="обст">здесь</W>
 <W DOM="2" FEAT="S ЕД МУЖ ТВОР ОД" ID="5" KSNAME="РУССКИЙ2" LEMMA="РУССКИЙ" LINK="2-компл">русским</W>, 
 <W DOM="5" FEAT="V СОВ СТРАД ПРИЧ ПРОШ ЕД МУЖ ТВОР" ID="6" LEMMA="ТЕРЯТЬ" LINK="опред">потерянным</W>
 <W DOM="6" FEAT="CONJ" ID="7" KSNAME="И1" LEMMA="И" LINK="сочин">и</W>
 <W DOM="7" FEAT="V СОВ СТРАД ПРИЧ ПРОШ ЕД МУЖ ТВОР" ID="8" KSNAME="ОТРЫВАТЬ1" LEMMA="ОТРЫВАТЬ" LINK="соч-союзн">оторванным</W>
 <W DOM="8" FEAT="PR" ID="9" LEMMA="ОТ" LINK="2-компл">от</W>
 <W DOM="9" FEAT="S МН МУЖ РОД НЕОД" ID="10" LEMMA="КОРЕНЬ" LINK="предл">корней</W>…
 </S>,
 <S ID="10">
 <W DOM="2" FEAT="A ЕД СРЕД ИМ" ID="1" KSNAME="ЭТОТ" LEMMA="ЭТОТ" LINK="опред">Это</W>
 <W 

In [42]:
cur_sent = sents[1]
num_left = 2
num_right = 2

left = []
right = []
for i, prev_ in enumerate(cur_sent.previous_siblings):
    if prev_ is not None and not isinstance(prev_, str):
        left.append(prev_)
        
    if len(left) >= num_left or prev_ is None:
        break

for i, next_ in enumerate(cur_sent.next_siblings):
    if next_ is not None and not isinstance(next_, str):
        right.append(next_)
        
    if len(right) >= num_right or next_ is None:
        break

left_context = '' if not left else ' '.join(get_text_from_sentence(prev_) for prev_ in left)
right_context = '' if not right else ' '.join(get_text_from_sentence(next_) for next_ in right)

In [43]:
cur_sent['ID']

'2'

In [44]:
left_context

'Мария Орлова'

In [45]:
right_context

'Вот уже полтора года живу я в этом красивом, зеленом курортном, провинциальном, холодном и чопорном городишке. "Здесь русский дух…"'

In [32]:
res, tokens, text = parse_sent(sents[45], get_pos_to_feats_format(), return_tokens_list_text=True)

In [33]:
res

[]

In [34]:
tokens

Sentence([Word({'form': 'лошадей', 'id': 1, 'dom': 2, 'link': 'предик', 'lemma': 'лошадь', 'feat': {'gender': 'жен', 'number': 'мн', 'case': 'род', 'animacy': 'од'}, 'pos': 'S'}, children={1: [Word({'form': 'породистых', 'id': 3, 'dom': 1, 'link': 'оп-опред', 'lemma': 'породистый', 'feat': {'number': 'мн', 'case': 'род'}, 'pos': 'A'}, children={1: [Word({'form': 'и', 'id': 4, 'dom': 3, 'link': 'сочин', 'lemma': 'и', 'feat': {}, 'pos': 'CONJ'}, children={1: [Word({'form': 'очень', 'id': 6, 'dom': 4, 'link': 'соч-союзн', 'lemma': 'очень', 'feat': {}, 'pos': 'ADV'}, children={1: [Word({'form': 'не', 'id': 5, 'dom': 6, 'link': 'огранич', 'lemma': 'не', 'feat': {}, 'pos': 'PART'}, children={})]})], 2: [Word({'form': 'не', 'id': 5, 'dom': 6, 'link': 'огранич', 'lemma': 'не', 'feat': {}, 'pos': 'PART'}, children={})]})], 2: [Word({'form': 'очень', 'id': 6, 'dom': 4, 'link': 'соч-союзн', 'lemma': 'очень', 'feat': {}, 'pos': 'ADV'}, children={1: [Word({'form': 'не', 'id': 5, 'dom': 6, 'link': '

In [20]:
def get_text_from_sentence(sent: bs4.element.Tag) -> str:
    return (''.join(fragment for fragment in sent.strings).strip()
        .replace('\n', ' ').replace('  ', ' ')
    )

In [21]:
def parse_feat(
    feat: str,
    pos_to_feats_formats: Dict[str, Dict[str, List[str]]],
) -> Dict[str, str]:
    result = {}
    
    features = feat.split()
    pos = features[0]
    
    result['pos'] = pos
    if pos not in pos_to_feats_formats:
        if len(features) > 1:
            print(f'`{pos}` not in pos_to_feats_formats, '
                  f'but there are {len(features) - 1} values unparsed: '
                  f'`{features[1:]}`'
            )
        
        return result
    
    feats_left_str = ' '.join(features[1:]).lower()
    for param, possible_values in pos_to_feats_formats[pos].items():

        for value in possible_values:
            value_regex = rf'\b{value}\b'
            value_match = re.search(value_regex, feats_left_str)

            if value_match:
                result[param] = value_match.group(0)
                feats_left_str = feats_left_str.replace(value_match.group(0), '')
                break
                
    if feats_left_str.replace(' ', '') != '':  # are there only spaces left?
        print(f"all params from pos_to_feats_formats were parsed, "
              f"but there is something left in string `{feats_left_str.strip()}` "
              f"orig `{feat}`"
        )
    
    return result

In [22]:
class Word(object):
    def __init__(
        self, word_dict: Dict[str, str], children: Optional[Dict[int, List['Word']]] = None,
        mandatory_attributes = ['dom', 'id']
    ):
        if any(attr not in word_dict for attr in mandatory_attributes):
            raise ValueError(f'no `dom` or `id` in word_dict `{word_dict}`')
        
        self.attrs = ['dom', 'feat', 'id', 'lemma', 'link', 'form']
        self.id = word_dict['id']
        self.dom = word_dict['dom']
        link = word_dict.get('link')
        self.link = link if link else '_root'
        self.form = word_dict['form']
        self.lemma = word_dict['lemma']
        
        # TODO: add condition to only do the search for non-verbs?
        #   (to prevent `bylo` in `(kto) by to ni bylo` from messing up `bylo` verb)
        if (any(
                re.search(rf'\b{modif}\b', self.lemma)
                for modif in PRON_ONLY_MODIFIERS
            ) or any(self.lemma == pron for pron in PRONS)
        ):
            self.pos = 'PRON'
        else:
            self.pos = word_dict['pos'] 
        
        self.feat = word_dict['feat']
        if 'ksname' in word_dict:
            self.ksname = word_dict['ksname']
            self.attrs.append('ksname')
        if 'nodetype'in word_dict:
            self.nodetype = word_dict['nodetype']
            
        if any(attr not in self.__dict__ for attr in word_dict):
            print(f'word_dict not parsed fully: {word_dict}')
        
        self.children: Dict[int, List['Word']] = {}
        if children:
            self.children = children
        self.parent = None
    
    
    def add_child(self, child: 'Word', depth: int):
        self.children.setdefault(depth, []).append(child)
        if depth == 1:
            child.parent = self
        if self.parent:
            self.parent.add_child(child, depth+1) 
       
    
    def filter_out_appos_groups(self) -> Dict[int, List['Word']]:
        # TODO: exclude groups that depend on target word but are linearly away
        full_group = self.group
        children = self.children 
        
        words_in_appos_groups = list(chain(
            *[appos_head.group
              for appos_head in full_group
              if appos_head.link == 'оп-опред']
        ))
        
        # NEW
#         potential_distant_dependents = list(chain(
#             *[dep_head.group
#               for dep_head in direct_dependents[0]
#               if dep_head.id != self.id + 1
#              ]
#         ))
#         potential_distant_dependents = []
#         for pot_dist_dep in potential_distant_dependents:
#             if pot_dist
        # NEW: save whether word group has appositive dependents
        if not words_in_appos_groups:
            self.has_appos = 0
            return children

        else:
            self.has_appos=1
            filtered_children_dict = {}
            for depth, children in self.children.items():
                for child in children:
                    if child in words_in_appos_groups:
                        continue
                    filtered_children_dict.setdefault(depth, []).append(child)

            return filtered_children_dict
    
    
    def get_children_depth(self, exclude_appos_groups=False):
        if exclude_appos_groups:
            children = self.filter_out_appos_groups()
        else:
            children = self.children
        return max(children.keys(), default=0)    
    children_depth = property(get_children_depth)
    
    def get_children_depth_no_appos(self):
        return self.get_children_depth(exclude_appos_groups=True)
    no_appos_children_depth = property(get_children_depth_no_appos)
    
    
    def get_group(self, depth=None, add_root=True, exclude_appos_groups=False):
        if exclude_appos_groups:
            children_dct = self.filter_out_appos_groups()
        else:
            children_dct = self.children
        if depth:
            children = list(chain(*[children_dct[d] for d in children_dct.keys() if d <= depth]))
        else:
            children = list(chain(*children_dct.values()))        
        if add_root:
            children += [self]
        return children
    group = property(get_group)
    
    
    def get_group_str(self, **kwargs):
        group = self.get_group(**kwargs)
        try:
            group_str = ' '.join([
                child.form
                for child in sorted(group, key = lambda word: int(getattr(word, 'id')))
            ])
        except (TypeError) as e:
            print(self, *group, sep='\n')
            raise(e)
            
        return group_str
    group_str = property(get_group_str) 
    
    def get_group_no_appos(self, **kwargs):
        return self.get_group(exclude_appos_groups=True, **kwargs)
    no_appos_group = property(get_group_no_appos)
    
    def get_group_str_no_appos(self, **kwargs):
        return self.get_group_str(exclude_appos_groups=True, **kwargs)
    no_appos_group_str = property(get_group_str_no_appos)
    
    
    def to_dict(self):
        # можно бы и word_dict...
        return {attr: getattr(self, attr) for attr in self.attrs}
    
    
    def __repr__(self):
        return (
            f"""Word({{'form': '{self.form}', 'id': {self.id}, """
            f"""'dom': {self.dom}, 'link': '{self.link}', """
            f"""'lemma': '{self.lemma}', 'feat': {self.feat}, """
            f"""'pos': '{self.pos}'}}, children={self.children})"""
        )
    
    
    def __str__(self):
        children = chain(*self.children.values())
        parent = self.parent
        return (
            f"""<Word({{'form': '{self.form}', 'id': {self.id}, """
            f"""'dom': {self.dom}, 'link': '{self.link}', 'pos': '{self.pos}' """
            f"""'lemma': '{self.lemma}', 'feat': {self.feat}}}, """
            f"""children={[(child.id, child.form) for child in children]}), """
            f"""parent={None if not parent else (parent.id, parent.form)}>"""
        )

In [23]:
class TokensList(list):
    def filter(self, **kwargs):
        if not kwargs:
            return 
        
        res = TokensList()
        
        def get_val(attr, word):
            if 'feat__' in attr:
                return getattr(word, 'feat').get(attr.split('__')[1])
            else:
                return None or getattr(word, attr)

        for word in self:
            if all(get_val(attr, word) == value for attr, value in kwargs.items()):
                res.append(word)        
        return res    
    
    def __repr__(self):
        list_repr = super().__repr__()
        return f'{self.__class__.__name__}({list_repr})'
    

class Sentence(TokensList):
    pass

In [24]:
def make_tree(
    root: Word,
    depth: int,
    words: List[Word],    
) -> None:    
    children = [word for word in words if word.dom == root.id]

    for word in children:
        root.add_child(word, 1)
        make_tree(word, 1, words)

In [25]:
def make_full_tree(
    words: List[WordDict],
) -> None:  
    
#     for word in words:
#         id_to_word[word.id] = word
    
    root = next(word for word in words if word.dom == '_root')
    words.root = root
    
    depth = 1
    make_tree(root, depth, words)

In [29]:
def get_pos_chain(
    words: List, include_feat=False, include_feat_name=False,
    word_sep=' ', pos_feat_sep='+', feat_sep='|',
    feat_name_val_sep = '=',
) -> str:
    
    word_strs = []
    for word in words:
        word_str = word.pos
        if include_feat and include_feat_name:
            feat_str = feat_sep.join(
                [f'{feat}{feat_name_val_sep}{val}'
                for feat, val in word.feat.items()]
            )
        elif include_feat:
            feat_str = feat_sep.join(word.feat.values())
        else:
            feat_str = ''
            
        word_str += f'{pos_feat_sep}{feat_str}' if feat_str else ''
        word_strs.append(word_str)
        
    return word_sep.join(word_strs)

In [30]:
def feats_dict_to_str(feats_dict: Dict[str, Union[str, int]]) -> str:
    return '|'.join([f'{feat}={val}' for feat, val in feats_dict.items()])

In [31]:
def has_relative_clause(word_group: List[Word]) -> bool:
    has_wh_relcl = any(word.link == 'релят' for word in word_group)
    has_participle = False
    
    # тоже можно изменить на any
    for word in word_group:
        if word.pos == 'V' and word.feat.get('representation') == 'прич':
            has_participle = True
            break
    has_relcl = has_wh_relcl or has_participle
    return has_relcl, has_wh_relcl, has_participle

In [57]:
def get_data_from_sent(
    full_analysis: str, text: str, words: Sentence,
    sent_number: int, context: Tuple[str],
    exclude_appos_descriptions=True
) -> List[Res]:
    res = []
    
    sent_chain = get_pos_chain(words)
    
    for verb in words.filter(pos='V'):
        dos = words.filter(dom=verb.id, feat__case='вин', link='1-компл')
        secos = words.filter(dom=verb.id, link='2-компл')
        
        if not (dos and secos):
            continue
        
        has_many_dos, has_many_ios = 0, 0
        if len(dos) >= 2:
            has_many_dos = 1
            print(len(dos))
        if len(secos) >= 2:
            has_many_ios = 1
            print(len(secos))
        do = dos[0]
        seco = secos[0]
        
        if exclude_appos_descriptions:
            do_group, do_group_str = do.no_appos_group, do.no_appos_group_str
            seco_group, seco_group_str = seco.no_appos_group, seco.no_appos_group_str
            do_children_depth = do.no_appos_children_depth
            seco_children_depth = seco.no_appos_children_depth
        else:
            do_group, do_group_str = do.group, do.group_str
            seco_group, seco_group_str = seco.group, seco.group_str
            do_children_depth = do.children_depth
            seco_children_depth = seco.children_depth
        
        do_had_appos = do.__dict__.get('has_appos', '')
        seco_had_appos = seco.__dict__.get('has_appos', '')
        
        (do_has_relcl, do_has_wh_relcl, 
         do_has_participle) = has_relative_clause(do_group)
        (seco_has_relcl, seco_has_wh_relcl, 
         seco_has_participle) = has_relative_clause(seco_group)
        
        do_pos_feat_chain = get_pos_chain(
            do_group, include_feat=True, include_feat_name=True
        )
        seco_pos_feat_chain = get_pos_chain(
            seco_group, include_feat=True, include_feat_name=True
        )
        
        verb_feats = feats_dict_to_str(verb.feat)
        verb_has_neg = 1 if words.filter(dom=verb.id, pos='PART', lemma='не') else 0
        
        seco_head_pos = 'PR' if seco.pos == 'PR' else 'V'
        if seco_head_pos == 'PR':
            seco_head = seco.form
            seco = words.filter(dom=seco.id, link='предл')[0]
        else:
            seco_head = verb.form
        
        id_to_role = {do.id: 'DO', verb.id: 'V', seco.id: 'IO'}
        order = sorted(
            [verb] + [do, seco],
            key=lambda token: token.id,
        )
        roles_order = ' '.join(id_to_role[token.id] for token in order)
        
        has_nodetype_in_our_forms = 0
        if any('nodetype' in word.__dict__ for word in chain(*[[verb], do_group, seco_group])):
            has_nodetype_in_our_forms = 1
        
        res.append(Res(
            sent_number, context[0], text, context[1],
                full_analysis, sent_chain,
            has_many_dos, has_many_ios,
            has_nodetype_in_our_forms,
            verb.lemma, verb.form,
            verb_feats, verb_has_neg,
            do_group_str, seco_group_str, 
                do_had_appos, seco_had_appos,
                do_has_relcl, seco_has_relcl,
                    do_has_wh_relcl, seco_has_wh_relcl,
                    do_has_participle, seco_has_participle,
                do_children_depth, seco_children_depth,
                    len(do_group), len(seco_group),
                do_pos_feat_chain, seco_pos_feat_chain,
            do.form,
            do.pos, do.lemma, do.feat.get('animacy', ''),
                do.feat.get('case', ''), do.feat.get('number', ''),
            seco.form,
            seco.pos, seco_head, seco_head_pos, seco.lemma,
                seco.feat.get('animacy', ''), seco.feat.get('case', ''),
                seco.feat.get('number', ''),
            roles_order
        ))
        
    return res 

In [53]:
def get_context(
    sent: bs4.element.Tag,
    left_context_len = 2,
    right_context_len = 2,
):
    left = []
    right = []
    for i, prev_ in enumerate(sent.previous_siblings):
        if prev_ is not None and not isinstance(prev_, str):
            left.append(prev_)

        if len(left) >= num_left or prev_ is None:
            break

    for i, next_ in enumerate(sent.next_siblings):
        if next_ is not None and not isinstance(next_, str):
            right.append(next_)

        if len(right) >= num_right or next_ is None:
            break

    left_context = '' if not left else ' '.join(get_text_from_sentence(prev_) for prev_ in left)
    right_context = '' if not right else ' '.join(get_text_from_sentence(next_) for next_ in right) 
    
    return left_context, right_context

In [52]:
def parse_sent(
    sent: bs4.element.Tag,
    pos_to_feats_formats: Dict[str, Dict[str, List[str]]],
    return_tokens_list_text: bool = False,
    return_context: bool = True,
    left_context_len: int = 2, right_context_len: int = 2,
    word_tag: str = 'W',
    data_from_sent_kwargs={},
):
    
    words = Sentence()
    for word in sent.find_all(word_tag, recursive=False):
        word_res = {}
        for attr, value in word.attrs.items():
            attr = attr.lower()
            if attr not in ['dom', 'id'] or value == '_root':
                word_res[attr] = value
            else:
                word_res[attr] = int(value)
                
#         word_res = {
#             attr.lower(): (value if (attr not in ['DOM', 'ID'] or value == '_root') else int(value))
#             for attr, value in word.attrs.items()
#         }
        word_res['lemma'] = word_res.get('lemma', '').lower()
        word_res['feat'] = parse_feat(word_res['feat'], pos_to_feats_formats)
        word_res['pos'] = word_res['feat'].pop('pos')
        word_form = word.string or ''
        word_res['form'] = word_form.lower()
        word_obj = Word(word_res)
        
        words.append(word_obj)
    
    make_full_tree(words)
    sent_number = sent.get('ID')
    text = get_text_from_sentence(sent)
    
    left_context, right_context = '', ''
    if return_context:
        left_context, right_context = get_context(sent, left_context_len, right_context_len)
    context = (left_context, right_context)

    res = get_data_from_sent(str(sent), text, words, sent_number, context, **data_from_sent_kwargs)
    if not return_tokens_list_text:
        return res
    else:
        return res, words, text

In [35]:
file = syntagrus_dir / Path('2019/baden-baden.tgt')

In [36]:
file_soup = get_soup(file).find('body')

In [37]:
sents = file_soup.find_all('S')

In [58]:
res, tokens, text = parse_sent(sents[9], get_pos_to_feats_format(), return_tokens_list_text=True)

In [59]:
res

[res(sent_num_file='10', left_context='Он чувствовал себя здесь русским, потерянным и оторванным от корней… Тургенев в течение семи лет бродил тут с собакой по окрестным лесам и горам, искал, думал, страдал.', text='Это чувство родства с ним дает мне силы жить здесь.', right_context='Элементы русской архитектуры - резные деревянные балкончики и наличники - напоминают Абрамцево, белоснежные и бледно-розовые усадьбы, заросшие сиренью, возвращают меня в нашу тихую подмосковную жизнь. Из влажной туманной зимы, из голубой мглы и одиночества Баден в апреле возрождается безграничными полями огромных сиреневых крокусов и пением птиц.', full_analysis='<S ID="10">\n<W DOM="2" FEAT="A ЕД СРЕД ИМ" ID="1" KSNAME="ЭТОТ" LEMMA="ЭТОТ" LINK="опред">Это</W>\n<W DOM="6" FEAT="S ЕД СРЕД ИМ НЕОД" ID="2" LEMMA="ЧУВСТВО" LINK="предик">чувство</W>\n<W DOM="2" FEAT="S ЕД СРЕД РОД НЕОД" ID="3" LEMMA="РОДСТВО" LINK="1-компл">родства</W>\n<W DOM="3" FEAT="PR" ID="4" KSNAME="С3" LEMMA="С" LINK="1-компл">с</W>\n<W 

In [60]:
print(*tokens, sep='\n')

<Word({'form': 'это', 'id': 1, 'dom': 2, 'link': 'опред', 'pos': 'PRON' 'lemma': 'этот', 'feat': {'gender': 'сред', 'number': 'ед', 'case': 'им'}}, children=[]), parent=(2, 'чувство')>
<Word({'form': 'чувство', 'id': 2, 'dom': 6, 'link': 'предик', 'pos': 'S' 'lemma': 'чувство', 'feat': {'gender': 'сред', 'number': 'ед', 'case': 'им', 'animacy': 'неод'}}, children=[(1, 'это'), (3, 'родства'), (4, 'с'), (5, 'ним')]), parent=(6, 'дает')>
<Word({'form': 'родства', 'id': 3, 'dom': 2, 'link': '1-компл', 'pos': 'S' 'lemma': 'родство', 'feat': {'gender': 'сред', 'number': 'ед', 'case': 'род', 'animacy': 'неод'}}, children=[(4, 'с'), (5, 'ним')]), parent=(2, 'чувство')>
<Word({'form': 'с', 'id': 4, 'dom': 3, 'link': '1-компл', 'pos': 'PR' 'lemma': 'с', 'feat': {}}, children=[(5, 'ним')]), parent=(3, 'родства')>
<Word({'form': 'ним', 'id': 5, 'dom': 4, 'link': 'предл', 'pos': 'PRON' 'lemma': 'он', 'feat': {'gender': 'муж', 'number': 'ед', 'case': 'твор', 'animacy': 'од'}}, children=[]), parent=(

In [61]:
token = tokens[1]
print(text, end='\n\n')
for attr in ('group_str', 'children_depth'):
    print(f"{attr}: {getattr(token, attr)}\n"
          f"{f'no_appos_{attr}'}: {getattr(token, f'no_appos_{attr}')}\n"
    )

attr = 'group'
print(
    f"len_{attr}: {len(getattr(token, attr))}\n"
    f"len_{f'no_appos_{attr}'}: {len(getattr(token, f'no_appos_{attr}'))}\n"
 )

Это чувство родства с ним дает мне силы жить здесь.

group_str: это чувство родства с ним
no_appos_group_str: это чувство родства с ним

children_depth: 3
no_appos_children_depth: 3

len_group: 5
len_no_appos_group: 5



In [62]:
def parse_sent_in_file_by_num(short_filename, sent_num, return_tokens_list=True):
    file = syntagrus_dir / Path(short_filename)
    file_soup = get_soup(file).find('body')
    
    sents = file_soup.find_all('S', recursive=False)
    sent = sents[sent_num-1]
    
    res, tokens = parse_sent(
        sents[182], get_pos_to_feats_format(),
        return_tokens_list=return_tokens_list
    )
    
    return res, tokens

In [63]:
gr = tokens.filter(id=5)[0].group_str
r = get_pos_chain(tokens.filter(id=5)[0].group, include_feat=True, include_feat_name=True)

print(gr, r, sep='\n')

ним
PRON+gender=муж|number=ед|case=твор|animacy=од


In [65]:
tokens[4]

Word({'form': 'ним', 'id': 5, 'dom': 4, 'link': 'предл', 'lemma': 'он', 'feat': {'gender': 'муж', 'number': 'ед', 'case': 'твор', 'animacy': 'од'}, 'pos': 'PRON'}, children={})

In [66]:
words = parse_sent(sentences[3], get_pos_to_feats_format())

NameError: name 'sentences' is not defined

In [ ]:
print(*words.filter(link='1-компл'), sep='\n')

In [ ]:
words[0].group_str

In [67]:
def do_search(files, num_first_files=None):
    total_passing, total_regex = 0, 0
    
    if num_first_files:
        files = files[:num_first_files]
    
    examples = {}
    files_dfs = []
    for file in tqdm(files):
        soup = get_soup(file)
        
        doc_body = soup.find('body')
        
        sents = find_sentences_do_io(doc_body)
        num_regex_sents = len(sents)
        
        unique_file_path = file.relative_to(syntagrus_dir)
        path_str = str(unique_file_path).replace('\\', '/')
        examples[path_str] = [str(sent) for sent in sents]
                        
        good_sents = []
        for sent in sents:
            parsed_sent = parse_sent(sent, get_pos_to_feats_format())
            if parsed_sent:
                good_sents.extend(parsed_sent)
                
        num_passing_sents = len(good_sents)
        print(f'file is `{unique_file_path}`, there are {num_passing_sents} passing sentences '
              f'(were {num_regex_sents} by regex)')
        total_passing += num_passing_sents
        total_regex += num_regex_sents
        
        good_sents_table = pd.DataFrame(good_sents)
        good_sents_table.insert(0, 'source', path_str)
        files_dfs.append(good_sents_table)
    
    print(f'passing {total_passing} among {total_regex} found by regex')
    return pd.concat(files_dfs), total_passing, total_regex

In [68]:
%%time
corpus_base, total_passing, total_regex = do_search(files)

file is `2003\anketa.tgt`, there are 16 passing sentences (were 26 by regex)
file is `2003\armeniya.tgt`, there are 6 passing sentences (were 15 by regex)
file is `2003\artist_mimansa.tgt`, there are 39 passing sentences (were 75 by regex)
file is `2003\ataka.tgt`, there are 1 passing sentences (were 3 by regex)
file is `2003\atlanty_i_atlantologi.tgt`, there are 15 passing sentences (were 22 by regex)
file is `2003\avtomatizatsiya.tgt`, there are 6 passing sentences (were 15 by regex)
file is `2003\a_on_myatezhnyi.tgt`, there are 20 passing sentences (were 35 by regex)
file is `2003\baklanov.tgt`, there are 54 passing sentences (were 82 by regex)
file is `2003\batareika.tgt`, there are 2 passing sentences (were 8 by regex)
file is `2003\bessonnitsa.tgt`, there are 6 passing sentences (were 13 by regex)
file is `2003\bez_epokhi.tgt`, there are 11 passing sentences (were 15 by regex)
file is `2003\biologiya.tgt`, there are 15 passing sentences (were 20 by regex)
file is `2003\bionika.tg

In [69]:
def cast_bool_cols_to_int(df: pd.DataFrame) -> pd.DataFrame:
    subword_bool_cols_to_int = ['has', 'is']
    pat_bool_cols_to_int = [rf'(\b|_){sub}(\b|_)' for sub in subword_bool_cols_to_int]

    for colname in df.columns:
        if any(re.search(pat, colname) for pat in pat_bool_cols_to_int):
            df.loc[:, colname] = df[colname].astype(int)
             

In [70]:
cast_bool_cols_to_int(corpus_base)

corpus = corpus_base.rename_axis('file_idx').reset_index()
corpus.loc[:, 'full_analysis'] = corpus['full_analysis'].str.replace('\n', ' | ')

In [71]:
path = Path('./syntagrus/result/syntagrus_6_1+context.csv')
path.parent.mkdir(parents=True, exist_ok=True)

#ID!
corpus.to_csv(path, decimal=',')

In [ ]:
# ### ТОЛЬКО НОВЫЕ КОЛОНКИ

# upd_cols = ['file_idx', 'source'] + NEW_COLS
# corpus_upd = corpus[upd_cols]
# corpus_upd.loc[:, 'file_source_idx'] = corpus_upd.apply(lambda row: f'{row.file_idx}_{row.source}', axis=1)

In [1]:
# path = Path('./syntagrus/result/syntagrus_4_newonly.csv')
# path.parent.mkdir(parents=True, exist_ok=True)
# corpus_upd.to_csv(path)

In [103]:
corpus[corpus['DO_has_relcl']==1]

,file_idx,source,sent_num_file,left_context,text,right_context,full_analysis,sent_pos_chain,has_many_dos,has_many_ios,...,DO_number,IO,IO_pos,IO_head,IO_head_pos,IO_lemma,IO_animacy,IO_case,IO_number,order
22,0,2003/artist_mimansa.tgt,4,"Это случилось вечером, в третьем акте. Илью Ил...","Будучи одет в малиновую ливрею с галунами, бел...","Он мирно стоял с подносом в кулисе № 2, ожидая...","<S ID=""4""> | <W DOM=""17"" FEAT=""V НЕСОВ ДЕЕПР Н...",V V PR A S PR S A S CONJ S PR S S S V V V PR S...,0,0,...,мн,балеринам,S,раздать,V,балерина,од,дат,мн,V IO DO
42,20,2003/artist_mimansa.tgt,237,От такого кошмара у Ильи Ильича выступила испа...,"Он пересиливал себя, вставал и ходил по комнат...","И все это было так ужасно, просто конец света....","<S ID=""237""> | <W DOM=""2"" FEAT=""S ЕД МУЖ ИМ ОД...",PRON V PRON V CONJ V PR S V PR S ADV V PRON S ...,0,0,...,ед,одеяло,S,под,PR,одеяло,неод,вин,ед,V IO DO
55,33,2003/artist_mimansa.tgt,368,"Еще двое вышли, но один вошел… Один вышел, ост...",Уж совсем было освободился директор - и тут на...,"Гости ушли через час, когда приемная уже ломил...","<S ID=""368""> | <W DOM=""4"" FEAT=""PART"" ID=""1"" K...",PART ADV PRON V S CONJ ADV V S A PR S PR S PR ...,0,0,...,ед,кабинет,S,в,PR,кабинет,неод,вин,ед,V IO DO
64,2,2003/atlanty_i_atlantologi.tgt,33,"В мае 2001 года в заливе Гуанаакабибес, на зап...",При горизонтальном сканировании глубоководные ...,Руководитель экспедиции Полина Зелицкая заявил...,"<S ID=""33""> | <W DOM=""6"" FEAT=""PR"" ID=""1"" KSNA...",PR A S A S V PR S S A A S A PR A S PR ADV A CO...,0,0,...,ед,дне,S,на,PR,дно,неод,пр,ед,V IO DO
68,6,2003/atlanty_i_atlantologi.tgt,54,Ищут следы Атлантиды не только в Западном полу...,"Так, в 1973 году американская подводная лодка ...",Американскому правительству было тотчас доложе...,"<S ID=""54""> | <W DOM=""9"" FEAT=""ADV"" ID=""1"" LEM...",ADV PR NUM S A A S ADV V PR S S S V S,0,0,...,мн,берегов,S,у,PR,берег,неод,род,мн,V IO DO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9508,9,uppsala/korp_726.tgt,35,"Следовательно, наши предки на Волыни и в Галич...",Святой равноапостольный князь Владимир Великий...,"Следовательно, и духовная власть упомянутого в...","<S ID=""35""> | <W DOM=""3"" FEAT=""A ЕД МУЖ ИМ"" ID...",A A S S A V S PR S PRON S S PRON V PART PART S...,0,0,...,ед,князем,S,назначил,V,князь,од,твор,ед,V IO DO
9519,5,uppsala/nagibin_1.tgt,46,"Он вернулся, вернулся сам, не дождавшись меня,...","- Обедать!.. - послышался голос мамы, и, груст...",Путешествие так и не состоялось. Но зато сколь...,"<S ID=""46""> | - <W DOM=""2"" FEAT=""V НЕСОВ ИНФ"" ...",V V S S CONJ ADV V S PRON ADV V PR V S PR A S ...,0,0,...,ед,взглядом,S,окинув,V,взгляд,неод,твор,ед,V IO DO
9549,35,uppsala/nagibin_1.tgt,270,"Но почему же началось это с Тенненбаума, котор...","Очень сильная, очень настоящая любовь делает п...","Вопреки очевидности, я где-то на дне души не в...","<S ID=""270""> | <W DOM=""2"" FEAT=""ADV"" ID=""1"" LE...",ADV A ADV A S V S PRON S PART PRON A PRON V PR...,0,0,...,ед,провидцем,S,делает,V,провидец,од,твор,ед,V IO DO
9552,2,uppsala/nagibin_2.tgt,15,"Все знали, что оно будет, это наступление, и ч...",Ради этого младший лейтенант Павлов скрывал от...,"Он был преступно недальновиден: думал, что рас...","<S ID=""15""> | <W DOM=""6"" FEAT=""PR"" ID=""1"" LEMM...",PR PRON A S S V PR PRON CONJ PR PRON S V PRON S,0,0,...,ед,себя,PRON,от,PR,себя,од,род,ед,V IO DO


In [206]:
corp = pd.read_csv('./syntagrus/result/syntagrus.csv')